In [ ]:
!pip install azure-ai-textanalytics==5.2.0b1
!pip install bs4
!pip install requests

In [71]:
# authenticate azure text analytics client
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential

key = "677e7a68d2f94932a97572bfd9505ea3"
endpoint = "https://covid-corruption-nlp2.cognitiveservices.azure.com/"

def authenticate_client():
    ta_credential = AzureKeyCredential(key)
    text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, 
            credential=ta_credential)
    return text_analytics_client

client = authenticate_client()


In [ ]:
# perform key phrase extraction
def key_phrase_extraction(client, documents=""):
    try:
        responses = client.extract_key_phrases(documents = documents)
        
        key_phrases = []
        for response in responses:
            if response.key_phrases:
                key_phrases += response.key_phrases
        
        return key_phrases

    except Exception as err:
        print("Encountered exception. {}".format(err))


news_source = "sky"
f_in = open('datasets/parsed-news/{}.txt'.format(news_source), 'r')
documents = [line.rstrip() for line in f_in.readlines()]
num_docs = len(documents)

key_phrases = []
for i in range(0, num_docs, 10):
    print(i)
    key_phrase = key_phrase_extraction(
                    client, 
                    documents=documents[i: min(num_docs, i+10)]
    )

    if key_phrase:
        key_phrases += key_phrase

print(key_phrases)
f_out = open('datasets/parsed-news/{}_phrases.txt'.format(news_source), 'w+')
for phrase in key_phrases:
    phrase = '-'.join(phrase.split())
    f_out.write(phrase + '\n')

In [6]:
# opinion mining on trustpilot reviews
from bs4 import BeautifulSoup
import requests
import pandas as pd

grouped_details = pd.read_csv('datasets/grouped_details.csv')
link_cols = [x for x in grouped_details.columns if 'company_link' in x]

def get_reviews(x):
    results = []
    for cell in x:
        if pd.isna(cell) or not cell:
            results.append('')
        else:
            page = requests.get("https://uk.trustpilot.com/review/"+cell)
            print(page)
            soup = BeautifulSoup(page.content, "html.parser")
            reviewcontent = soup.find_all('p', {'class', 'review-content__text'})
            
            results.append([review.get_text() for review in reviewcontent if review])
    return results

link_details = pd.DataFrame(grouped_details[link_cols])
tr_reviews = link_details.apply(get_reviews)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
<Response [200]>
<Response [404]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [404]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

In [7]:
tr_reviews.to_csv('reviews.csv')


,company_link_0,company_link_1,company_link_2,company_link_3
0,[\n I saw many bad reviews for ...,,,
1,[],,,
2,[\n Appalling service provider ...,,,
3,[],,,
4,[],,,
...,...,...,...,...
708,[],[],,
709,[\n Stella was amazing in our t...,,,
710,[\n Stella was amazing in our t...,,,
711,"[\n Rapid results, great custom...",[],,


In [ ]:
def opinion_mining(client,documents):

    result = client.analyze_sentiment(documents, show_opinion_mining=True)
    doc_result = [doc for doc in result if not doc.is_error]

    negative_keywords=[]
    for document in doc_result:
        for sentence in document.sentences:
            for mined_opinion in sentence.mined_opinions:
                if mined_opinion.target.sentiment == 'negative':
                    negative_keywords.append(mined_opinion.assessments[0].text)
                else:
                    negative_keywords.append('')
    return negative_keywords 

tr_reviews =  pd.read_csv('reviews.csv')
keywords = []
for review in tr_reviews['company_link_0']:
    #print(type(review))
    if type(review) == str: 
        review = eval(review)
        num_docs = len(review)
        for i in range(0, num_docs, 10):
            keywords.append(opinion_mining(client,documents=review[i: min(num_docs, i+10)]))

In [100]:
keywords = pd.DataFrame(keywords)
keywords['company_name'] = grouped_details['company_name']
keywords

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,company_name
0,,,cheapest,,shocking,false,breach,use,,,...,None,None,None,None,None,None,None,None,None,#nomad travel
1,,,harder,,,,work,rude,,,...,None,None,None,None,None,None,None,None,None,*everything genetic ltd
2,shame,scam,false,match,match,lost,trusted,poor,working,recognise,...,None,None,None,None,None,None,None,None,None,+44 covid test
3,,inefficient,,,limited,use,slimy,poor,shoddy,poor,...,None,None,None,None,None,None,None,None,None,.0000covid 19 private test ltd
4,use,appalling,,waiting,waiting,,,bad,registered,,...,None,None,None,None,None,None,None,None,None,.0001 ace-thetix
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
487,,,,,trusted,,,,,,...,None,None,None,None,None,None,None,None,None,nivha laboratory services ltd
488,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,"nivha laboratory services ltd, randox"
489,,,,,,,,,great,,...,None,None,None,None,None,None,None,None,None,nivha laboratory services ltd.
490,,,,,,,,,,,...,None,None,None,None,None,None,None,None,None,nl clinic


In [45]:
print(tr_reviews['company_link_0'][0][1])
#opinion_mining(client,tr_reviews['company_link_0'][0])


                Family member used this company after returning from a amber country. They use oncologica which has shocking reviews.Day 6 and still no sign of the day 2 results.   They refuse to recognise the Royal Mail delivery time and send a email when they scan the item. Which could be up to 12 hrs later. This gives them even longer to process the test. Still no results. False advertising, trading standards and breach of contract!
            
